In [1]:
import pandas as pd
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv("weatherHistory.csv")
dataset.head()

In [ ]:
dataset = dataset.drop(columns=["Formatted Date", "Summary", "Precip Type", "Loud Cover", "Daily Summary"])
dataset.head()

In [ ]:
features = dataset.drop(columns=["Humidity"])
target = dataset["Humidity"]

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(32, activation="relu", input_shape=(x_train.shape[1],)),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(1)
])

print(model.summary())

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[early_stopping])

In [ ]:
loss, mae = model.evaluate(x_test, y_test)
print("Loss:", loss)
print("MAE:", mae)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"], label="Training MAE")
plt.plot(history.history["val_mae"], label="Validation MAE")
plt.title("MAE Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(x_test[20:40])
for (x, y) in zip(predictions, y_test[20:40]):
    print(f"Predicted: {x[0]:.2f}, Actual: {y}")